In [94]:
from gensim.models import KeyedVectors, Word2Vec
from gensim.parsing.preprocessing import remove_stopwords
from tinydb import TinyDB, Query
import numpy as np
import datetime
import utils
import spacy
import re
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
# Load the polish Word2Vec model
word2vec = KeyedVectors.load('./word2vec_100_3_polish.bin')

In [118]:
db_mz_news = TinyDB('./data/mz_news.json')
db_mz_messages = TinyDB('./data/mz_messages.json')

db_wyborcza = TinyDB('./data/wyborcza.json')
db_tvpinfo = TinyDB('./data/tvpinfo.json')

In [114]:
q = Query()

In [134]:
db_mz_news.search(
    (q.release_date > datetime.datetime( year=2023, month=7, day=29 ).timestamp())
    &
    (q.release_date < datetime.datetime( year=2023, month=8, day=1  ).timestamp())
)

[{'title': 'Zmiana zasad przepisywania recept na leki psychotropowe i środki odurzające',
  'url': 'https://www.gov.pl/web/zdrowie/zmiana-zasad-przepisywania-recept-na-leki-psychotropowe-i-srodki-odurzajace',
  'release_date': 1690754400.0,
  'content': 'Wypisaniu recepty na leki psychotropowe i środki odurzające, od 2 sierpnia musi towarzyszyć badanie i ocena wpływu leku na pacjenta. Uwaga: mówimy tu o badaniu stacjonarnym ale też konsultacji przeprowadzonej z pacjentem zdalnie. Pamiętajmy, że konsultacja to obowiązkowa rozmowa z pacjentem na temat jego stanu zdrowia, stanu psychicznego oraz wywiad na temat zażywanych leków. Zasada ta nie dotyczy przepisywania recept w ramach kontynuacji leczenia, jeśli nie upłynęły \xa03 miesiące od ostatniej wizyty u lekarza i badania pacjenta. Informacja, o tym, że wskazany okres nie został przekroczony będzie zamieszczana\xa0 w dokumentacji medycznej pacjenta.\nPrzed wystawieniem każdej kolejnej recepty na środki odurzające i leki psychotropowe \x

In [15]:
nlp = spacy.load('pl_core_news_lg')

# Compute the mean computationally for each sentence in the text
def compute_mean(sentence):
    mean_v = np.zeros((100,))
    skipped = []
    for i in range(len(sentence)):
        word = sentence[i]
        word_lemma = nlp(word)[0].lemma_.lower()

        try:
            mean_v += (word2vec[word_lemma] - mean_v) / (i+1)
            #print(word_lemma)
        except KeyError:
            skipped.append(word)

    return mean_v

In [138]:
for article in db_tvpinfo.all():
    processed_article = utils.preprocess(article['content'])
    article_date = datetime.datetime.fromtimestamp(article['release_date'])

    # Find source articles from around the same time
    mz_news = db_mz_news.search(
        (q.release_date > (article_date - datetime.timedelta(days=2)).timestamp())
        &
        (q.release_date < article['release_date'])
    )
    if len(mz_news) == 0:
        continue

    for sentence in processed_article:
        if len(sentence) < 4:
            continue

        sentence_mean = compute_mean(sentence)

        # For every source article
        for source in mz_news:
            processed_source = utils.preprocess(source['content'])

            for source_sentence in processed_source:
                if len(source_sentence) < 4:
                    continue

                source_sentence_mean = compute_mean(source_sentence)

                cosine_sim = np.dot(sentence_mean, source_sentence_mean) / (np.linalg.norm(sentence_mean)*np.linalg.norm(source_sentence_mean))
                if cosine_sim>0.8:
                    print('%s |||| %s' % (sentence, source_sentence))
            

In [20]:
for news in db_mz_news.all():
    processed_news = utils.preprocess(news['content'])
    for sentence in processed_news:
        print(compute_mean(sentence))

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0.]
[ 9.85370942e-02 -9.03209906e-01 -1.97549604e+00  1.75369068e+00
  6.09800777e-01 -2.01759429e+00 -3.65361931e-01  4.23235391e-01
 -1.79486646e+00 -1.29184829e+00 -3.05793037e+00 -2.50434855e+00
 -6.50505917e-01 -2.62496671e-01 -2.98528077e-01 -1.10325518e+00
  9.26612900e-01  1.08758080e+00 -6.02749534e-01 -6.83232199e-01
 -1.16437677e+00  1.82512434e+00  8.55983396e-02  1.80385891e+00
 -3.44690670e-01  4.87487253e-01  4.18626143e+00 -1.04946902e+00
 -2.87132086e-01  5.00436458e-01  2.52781131e+00  7.78072114e-01
 -1.55022566e+00 -3.64194658e-01  1.60480284e+00  1.69577579e+00
 -2.73298117e+00  1.35153431e+00  2.62066282e+00 -1.11717330e-01
 -3.91694021e-02 -5.32896289e-02 -3.00496954

KeyboardInterrupt: 

In [92]:
x = utils.preprocess('Teraz Wirtualna Polska informuje, że w Platformie Obywatelskiej mówi się o tym, że Lenz w ostatnich miesiącach bardzo się starał i regularnie pojawiał się na spotkaniach z Tuskiem, próbując.')
y = utils.preprocess('Ukraina znowu zaatakowała most na Krymie, są straty w ludziach.')


In [93]:
x = compute_mean(x[0])
y = compute_mean(y[0])

In [87]:
np.dot(x, y) / (np.linalg.norm(x)*np.linalg.norm(y))

0.046070289975513636